In [2]:
import os
import json
import pandas as pd
import langchain
import ollama

In [3]:

from langchain.llms import Ollama
from langchain import PromptTemplate

In [4]:
llm = Ollama(model="orca-mini", temperature=0)

In [11]:
llm.invoke("Tell me a joke about cricket")

' Why did the cricket join the gym? To get a bat and ball!'

In [5]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}



In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [40]:
prompt=PromptTemplate(template=TEMPLATE,input_variables=["text","number","subject","tone","response_json"])

In [17]:
quiz_chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz",verbose=True)

In [38]:
responsestring=json.dumps(RESPONSE_JSON)


In [39]:
responsestring

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
#response=chain1.invoke({"text":"chemistry is very easy subject only molecules are present you can change them","number":1,"subject":"chemistry","tone":"simple","response_json": responsestring})



> Entering new LLMChain chain...
Prompt after formatting:

Text:chemistry is very easy subject only molecules are present you can change them
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 1 multiple choice questions for chemistry students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 1 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct":

In [42]:
response['response']

' Sure, here are 3 multiple-choice questions related to chemistry:\n\n1. Which of the following is a type of chemical reaction?\na) Combustion\nb) Synthesis\nc) Decomposition\nd) Single displacement\n\n2. Which of the following is not a property of an element?\na) Atomic number\nb) Chemical symbol\nc) Formula\nd) Melting point\n\n3. Which of the following is not a type of chemical bond?\na) Covalent\nb) Ionic\nc) Metallic\nd) Vaporous'

In [42]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [45]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [46]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [47]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [48]:
with open("C:/Users/vk/mcqgeneratorproject/biology.txt","r") as F:
    file=F.read()

In [29]:
print(file)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [60]:
NUMBER=10
SUBJECT="biology"
TONE="complex"

In [61]:
answer=generate_evaluate_chain.invoke({"text":file,"number":NUMBER,"subject":SUBJECT,"tone":TONE,"response_json":responsestring})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [63]:
answer

{'text': "Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [109]:
print(type(answer['quiz']))

<class 'str'>


In [110]:
answer['quiz']

' Here are 10 multiple-choice questions related to biology for students:\n\n1. Which of the following is not a type of cell?\na) Plant cell\nb) Animal cell\nc) Bacteria cell\nd) Fungi cell\nCorrect answer: b) Animal cell\n\n2. The process by which organisms produce offspring is called ____________.\na) Reproduction\nb) Development\nc) Metamorphosis\nd) Asexual reproduction\nCorrect answer: c) Metamorphosis\n\n3. Which of the following is not a characteristic of cells?\na) Cell membrane\nb) Cell wall\nc) Cellulose\nd) Lysosomes\nCorrect answer: d) Lysosomes\n\n4. The process by which organisms exchange genetic material is called ____________.\na) Reproduction\nb) Development\nc) Metamorphosis\nd) Asexual reproduction\nCorrect answer: b) Animal reproduction\n\n5. Which of the following is not a type of virus?\na) Plant virus\nb) Animal virus\nc) Bacteria virus\nd) Fungi virus\nCorrect answer: c) Bacteria virus\n\n6. The process by which organisms produce offspring is called ____________.

In [90]:
import reportlab

In [91]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [120]:
c=canvas.Canvas("quizanswers1.pdf",pagesize=letter)

In [121]:
c.drawString(200,750,answers)

In [122]:
c.save()

In [111]:
answers=answer['quiz']

In [123]:
print(answers)

 Here are 10 multiple-choice questions related to biology for students:

1. Which of the following is not a type of cell?
a) Plant cell
b) Animal cell
c) Bacteria cell
d) Fungi cell
Correct answer: b) Animal cell

2. The process by which organisms produce offspring is called ____________.
a) Reproduction
b) Development
c) Metamorphosis
d) Asexual reproduction
Correct answer: c) Metamorphosis

3. Which of the following is not a characteristic of cells?
a) Cell membrane
b) Cell wall
c) Cellulose
d) Lysosomes
Correct answer: d) Lysosomes

4. The process by which organisms exchange genetic material is called ____________.
a) Reproduction
b) Development
c) Metamorphosis
d) Asexual reproduction
Correct answer: b) Animal reproduction

5. Which of the following is not a type of virus?
a) Plant virus
b) Animal virus
c) Bacteria virus
d) Fungi virus
Correct answer: c) Bacteria virus

6. The process by which organisms produce offspring is called ____________.
a) Reproduction
b) Development
c) Met